In [2]:
# Load libraries
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

In [6]:
energy_con = pd.read_csv(r"\Your.csv", )

# Transformation to extract related data
relatable_cols = [
  
    'country', 'year', 

    'population', 'gdp',

    'primary_energy_consumption', 'energy_per_capita', 'energy_per_gdp',

    'renewables_consumption', 'renewables_share_energy', 'renewables_share_elec',

    'fossil_fuel_consumption', 'fossil_share_energy',

    'coal_share_energy', 'oil_share_energy', 'gas_share_energy',
    'nuclear_share_energy', 'biofuel_share_energy', 'hydro_share_energy',
    'solar_share_energy', 'wind_share_energy',

    'coal_consumption', 'oil_consumption', 'gas_consumption',
    'nuclear_consumption', 'hydro_consumption', 'solar_consumption', 'wind_consumption',

    'greenhouse_gas_emissions', 'carbon_intensity_elec',

    'energy_cons_change_pct', 'renewables_cons_change_pct', 'fossil_cons_change_pct'
]

energy_con = energy_con[relatable_cols].dropna(subset=['country', 'year'])
 
europe_countries = [
       "Austria", "Belgium",  
    "Bulgaria", "Croatia",   "Czechia", "Denmark", "Estonia", "Finland",
    "France", "Germany", "Greece", "Hungary",   "Ireland", "Italy",
      "Latvia",  "Lithuania",  
        "Netherlands", "North Macedonia", "Norway",
    "Poland", "Portugal", "Romania",    "Slovakia",
    "Slovenia", "Spain", "Sweden", "Switzerland",  
    "United Kingdom"
]

energy_con = energy_con[energy_con["country"].isin(europe_countries)]

energy_con = energy_con[
    (energy_con["year"] >= 2000) & (energy_con["year"] <= 2020)
]

In [8]:
# List of columns to impute (excluding identifiers)
columns_to_impute = [
    'population', 'gdp',
    'primary_energy_consumption', 'energy_per_capita', 'energy_per_gdp',
    'renewables_consumption', 'renewables_share_energy', 'renewables_share_elec',
    'fossil_fuel_consumption', 'fossil_share_energy', 
    'coal_share_energy', 'oil_share_energy', 'gas_share_energy',
    'nuclear_share_energy', 'biofuel_share_energy', 'hydro_share_energy',
    'solar_share_energy', 'wind_share_energy', 
    'coal_consumption', 'oil_consumption', 'gas_consumption',
    'nuclear_consumption', 'hydro_consumption', 'solar_consumption', 'wind_consumption',
    'greenhouse_gas_emissions', 'carbon_intensity_elec',
    'energy_cons_change_pct', 'renewables_cons_change_pct', 'fossil_cons_change_pct'
]

def interpolate_group(df_group):
    # Ensure relevant columns are numeric to avoid dtype issues
    df_numeric = df_group[columns_to_impute].apply(pd.to_numeric, errors='coerce')
    df_numeric = df_numeric.interpolate(method='linear', limit_direction='both')
    # Prepare a copy to not modify original group
    df_out = df_group.copy()
    # Replace only imputed columns
    df_out.loc[:, columns_to_impute] = df_numeric
    return df_out.sort_values('year')

energy_con = energy_con.groupby('country', group_keys=False).apply(interpolate_group)

C:\Users\Elijah\AppData\Local\Temp\ipykernel_24152\2508472475.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  energy_con = energy_con.groupby('country', group_keys=False).apply(interpolate_group)


In [49]:
# Checkin the result of imputation
energy_con.info()
print(energy_con[columns_to_impute].isnull().sum())

# nas are still present due to missing observations in the entire period for some countries e.g. austria without nuclear plants lacks nuclear_share_energy_data. In the next steps of the project (PowerBI) will be imputed with zero's.

<class 'pandas.core.frame.DataFrame'>
Index: 588 entries, 1777 to 20585
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   country                     588 non-null    object 
 1   year                        588 non-null    int64  
 2   population                  588 non-null    float64
 3   gdp                         588 non-null    float64
 4   primary_energy_consumption  588 non-null    float64
 5   energy_per_capita           588 non-null    float64
 6   energy_per_gdp              588 non-null    float64
 7   renewables_consumption      588 non-null    float64
 8   renewables_share_energy     588 non-null    float64
 9   renewables_share_elec       588 non-null    float64
 10  fossil_fuel_consumption     588 non-null    float64
 11  fossil_share_energy         588 non-null    float64
 12  coal_share_energy           588 non-null    float64
 13  oil_share_energy            588 non

In [50]:
# Save Energy data
energy_con.to_csv('energy_data.csv', index=False)

In [22]:
tourism = pd.read_csv(r"\Your.csv", )
tourism.drop('country_code', axis=1, inplace=True)
europe_countries_t = [
       "Austria", "Belgium",  
    "Bulgaria", "Croatia",   "Czechia", "Denmark", "Estonia", "Finland",
    "France", "Germany", "Greece", "Hungary",   "Ireland", "Italy",
      "Latvia",  "Lithuania",  
        "Netherlands", "North Macedonia", "Norway",
    "Poland", "Portugal", "Romania",    "Slovak Republic",
    "Slovenia", "Spain", "Sweden", "Switzerland",  
    "United Kingdom"
]

tourism = tourism[tourism["country"].isin(europe_countries_t)]

tourism.loc[tourism["country"] == "Slovak Republic", "country"] = "Slovakia"

tourism = tourism[
    (tourism["year"] >= 2000) & (tourism["year"] <= 2020)
    ]
tourism = tourism.sort_values(by=['country', 'year'], ascending=[True, True])

In [24]:
# List of columns to impute (excluding identifiers)
columns_to_impute_tourism = [            
     'tourism_receipts',   
     'tourism_arrivals',       
     'tourism_exports',        
     'tourism_departures',     
     'tourism_expenditures',  
    'gdp' , 'inflation', 'unemployment'      
]

def interpolate_group_t(df_group_t):
    # Ensure relevant columns are numeric to avoid dtype issues
    df_numeric_t = df_group_t[columns_to_impute_tourism].apply(pd.to_numeric, errors='coerce')
    df_numeric_t = df_numeric_t.interpolate(method='linear', limit_direction='both')
    # Prepare a copy to not modify original group
    df_out_t = df_group_t.copy()
    # Replace only imputed columns
    df_out_t.loc[:, columns_to_impute_tourism] = df_numeric_t
    return df_out_t.sort_values('year')

tourism = tourism.groupby('country', group_keys=False).apply(interpolate_group_t)

C:\Users\Elijah\AppData\Local\Temp\ipykernel_24152\226122925.py:21: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tourism = tourism.groupby('country', group_keys=False).apply(interpolate_group_t)


In [25]:
tourism.info()

<class 'pandas.core.frame.DataFrame'>
Index: 588 entries, 280 to 5667
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   country               588 non-null    object 
 1   year                  588 non-null    int64  
 2   tourism_receipts      546 non-null    float64
 3   tourism_arrivals      588 non-null    float64
 4   tourism_exports       546 non-null    float64
 5   tourism_departures    567 non-null    float64
 6   tourism_expenditures  546 non-null    float64
 7   gdp                   588 non-null    float64
 8   inflation             588 non-null    float64
 9   unemployment          588 non-null    float64
dtypes: float64(8), int64(1), object(1)
memory usage: 50.5+ KB


In [ ]:
# Save Tourism data
tourism.to_csv('tourism_data.csv', index=False)